# DicoCo

*antoine LEPERCQ*

In [1]:
# import necessaire à la création des widgets
import os
import re
import pandas as pd
import requests
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Markdown, Audio
from requests_html import HTMLSession
from bs4 import BeautifulSoup
#for loading and visualizing audio files
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np

os.chdir(r"D:\Antoine\CNAM\3A\WS\cnam-athon-pronunciation-main\data")

In [2]:
buchanan = pd.read_csv("dataframe\BigiRichSample.csv", sep=";", encoding="utf-8")

In [3]:
buchanan[buchanan.hwd.notnull()].head()

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,sizePron,motVedette,hwd,pronSampa,ipa
0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,7,abacus,abacus,{ b @ k @ s,æbəkəs
3,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,8,abandon,abandon,@ b { n d @ n,əbændən
4,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,10,abandoned,abandoned,@ b { n d @ n d,əbændənd
7,2,ăbăʹſh,F.,NaN,NaN,v.,9.0,19,6,abash,abash,@ b { S,əbæʃ
8,2,ăbāʹte,F.,NaN,NaN,v.,10.0,19,6,abate,abate,@ b eI t,əbeIt


In [4]:
def getAudioInfo(mot):
    session = HTMLSession()
    response = session.get("https://dictionary.cambridge.org/fr/dictionnaire/anglais/"+mot)
    html_doc = response.content
    soup = BeautifulSoup(html_doc, 'html.parser')
    words = soup.findAll("div", {"class": "pos-header dpos-h"})
    
    words_list = []
    
    for word in words:
        
        row = []

        #on prend toutes les infos necessaires (mp3, pos, ipa)
        audioAll = word.findAll(type="audio/mpeg")
        posAll = word.findAll('span', {'class': 'pos dpos'})
        ipaAll = word.findAll('span', {'class': 'ipa dipa lpr-2 lpl-1'})
        
        #on récupère pos et ipa
        row.append(posAll[0].text)
        row.append(ipaAll[0].text)
        
        #on récupère le chemin du mp3 selon la version uk ou us
        for a in audioAll:
            if "uk_pron" in a.get_attribute_list('src')[0]:
                row.append("https://dictionary.cambridge.org/"+a.get_attribute_list('src')[0])
            elif len(audioAll)==1:
                row.append("")
            
            if "us_pron" in a.get_attribute_list('src')[0]:
                row.append("https://dictionary.cambridge.org/"+a.get_attribute_list('src')[0])
            elif len(audioAll)==1:
                row.append("")
                
                
        words_list.append(row)
    
    df = pd.DataFrame.from_records(words_list, columns=["pos", "ipa", "audio_uk", "audio_us"])
    
    return df

In [5]:
# definition du widget permettant de taper un texte 
textTarget = widgets.Text(
       value="",
       description='Entrer un mot',
       layout={'width': '200px'} )

# definition du widget permettant de taper un texte pour l'ipa
ipaTarget = widgets.Text(
       value="",
       description='IPA',
       layout={'width': '200px'} )

# sélection d'une plage de valeurs
rangeSyllMin = 1
rangeSyllMax = 27
optionsNbSyll = range(1, 27)
wid_nbsyll = widgets.SelectionRangeSlider(
    options=optionsNbSyll,
    index=(min(optionsNbSyll)-1,max(optionsNbSyll)-1),
    description='Nb syllabes',
    disabled=False
)

nbResultMax = 20
wid_nbResult = widgets.IntSlider(
    value=nbResultMax,
    min=1,
    max=nbResultMax,
    step=1,
    description='Nb résultats max.',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [6]:
# définition du widget button permettant de lancer l'extraction
button = widgets.Button(description='Démarrer')

# définition du widget permettant l'affichage d'une sortie
cambridge = widgets.Output()
longman = widgets.Output()
linguee = widgets.Output()
tableau = widgets.Output()
    
def removeRegex(inputString):
    outputstring = ""
    for c in inputString:
        if c.isalpha():
            outputstring += c
    return outputstring

# spécifier les actions à réaliser lors de l'appui sur le bouton
def on_button_clicked(_):
    
    path = "D:/Antoine/CNAM/3A/WS/cnam-athon-pronunciation-main/data/audio/"

    # récupérer le texte saisi par l'utilisateur
    if len(textTarget.value) != 0:
        colToUse = 'hwd'
        targetWord = removeRegex(textTarget.value)
        
    else:
        colToUse = 'ipa'
    
    with cambridge:
        #Ne fonctionne pas corrctement avec le regex
        clear_output()
        urlCambridge = "https://dictionary.cambridge.org/fr/dictionnaire/anglais/"
        urlTarget = urlCambridge+targetWord
        display(HTML('<a href="'+urlTarget+'" target="_blank" >'+'Cambridge : '+targetWord+'</a>'))

    with longman:
        #Ne fonctionne pas corrctement avec le regex
        clear_output()        
        urlLongman = "https://www.ldoceonline.com/dictionary/"
        urlTarget = urlLongman+targetWord
        display(HTML('<a href="'+urlTarget+'" target="_blank" >'+'longman : '+targetWord+'</a>'))

    with linguee:
        #Ne fonctionne pas corrctement avec le regex
        clear_output()
        urlLinguee = "https://www.linguee.fr/anglais-francais/traduction/"
        urlTarget = urlLinguee+targetWord+'.html'
        display(HTML('<a href="'+urlTarget+'" target="_blank" >'+'linguee : '+targetWord+'</a>'))
        
    #filtrage tableau
    buchananFiltered = buchanan[buchanan[colToUse].str.contains(textTarget.value, na=False, regex=True)]
    buchananFiltered = buchananFiltered[buchananFiltered['nbSyll'].between(wid_nbsyll.value[0], wid_nbsyll.value[1], inclusive=True)]

    with tableau:
        clear_output()
        display(buchananFiltered[['motVedette', 'nbSyll', 'pron', 'pronSampa', 'ipa']].head(n=wid_nbResult.value))
           
    for wordInfo in buchananFiltered.head(n=wid_nbResult.value).itertuples():
        #TABLEAU
        getAudioInfo(wordInfo.motVedette).head()
        
        if getAudioInfo(wordInfo.motVedette).empty:
            print("pas d'informations pour", wordInfo.motVedette)
        
        else:
            row = [wordInfo.motVedette,
                   getAudioInfo(wordInfo.motVedette)[0:1].ipa.values[0],
                   getAudioInfo(wordInfo.motVedette)[0:1].pos.values[0],
                   "",
                   ""]
            
            #INFOS
            items = [widgets.Label(row[0].upper()+":"), widgets.Label(row[2]), widgets.Label(" - "+row[1])]
            display(widgets.HBox(items))
            
            #AUDIO - get_url
            if len(getAudioInfo(wordInfo.motVedette)[0:1].audio_uk)>0:
                row[3] = getAudioInfo(wordInfo.motVedette)[0:1].audio_uk.values[0]
                audio_uk = True
                #DOWNLOAD MP3
                session_uk = HTMLSession()
                audio = session_uk.get(row[3])
                with open(path+wordInfo.motVedette+'_uk.mp3', 'wb') as f:
                    f.write(audio.content)
            if len(getAudioInfo(wordInfo.motVedette)[0:1].audio_us)>0:
                row[4] = getAudioInfo(wordInfo.motVedette)[0:1].audio_us.values[0]
                audio_us = True
                #DOWNLOAD MP3
                session_us = HTMLSession()
                audio = session_us.get(row[4])
                with open(path+wordInfo.motVedette+'_us.mp3', 'wb') as f:
                    f.write(audio.content)
            
            #AUDIO - player
            if audio_uk and audio_us:
                audio_widgets = []
                for i in range(2):
                    if i == 0:
                        #widget audio
                        outAudio = widgets.Output()
                        with outAudio:
                            display(Audio(url=row[3]))
                        
                        #widget spectrogram
                        y, sr = librosa.load(path+wordInfo.motVedette+'_uk.mp3')
                        D = librosa.stft(y)  # STFT of y
                        S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
                        
                        outSpec = widgets.Output()
                        with outSpec:
                            fig, ax = plt.subplots()
                            img = librosa.display.specshow(S_db, x_axis='time', y_axis='linear', ax=ax)
                            ax.set(title='UK spectrogram')
                            fig.colorbar(img, ax=ax, format="%+2.f dB")
                            plt.show(fig)
                            
                        audio_widgets.append(widgets.VBox([widgets.Label("UK"), outAudio, outSpec]))
                        
                    if i == 1:
                        #widget audio
                        outAudio = widgets.Output()
                        with outAudio:
                            display(Audio(url=row[4]))
                            
                        #widget spectrogram
                        y, sr = librosa.load(path+wordInfo.motVedette+'_us.mp3')
                        D = librosa.stft(y)  # STFT of y
                        S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
                        
                        outSpec = widgets.Output()
                        with outSpec:
                            fig, ax = plt.subplots()
                            img = librosa.display.specshow(S_db, x_axis='time', y_axis='linear', ax=ax)
                            ax.set(title='US spectrogram')
                            fig.colorbar(img, ax=ax, format="%+2.f dB")
                            plt.show(fig)
                            
                        audio_widgets.append(widgets.VBox([widgets.Label("US"), outAudio, outSpec]))
                    
                display(widgets.HBox(audio_widgets))
                
            elif audio_uk:
                #widget audio
                outAudio = widgets.Output()
                with outAudio:
                    display(Audio(url=row[3]))
                    
                #widget spectrogram
                y, sr = librosa.load(path+wordInfo.motVedette+'_uk.mp3')
                D = librosa.stft(y)  # STFT of y
                S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
                    
                        
                outSpec = widgets.Output()
                with outSpec:
                    fig, ax = plt.subplots()
                    img = librosa.display.specshow(S_db, x_axis='time', y_axis='linear', ax=ax)
                    ax.set(title='UK spectrogram')
                    fig.colorbar(img, ax=ax, format="%+2.f dB")
                    plt.show(fig)

                display(widgets.VBox([widgets.Label("UK"), outAudio, outSpec]))   
                
                
            else:
                #widget audio
                out = widgets.Output()
                with out:
                    display(Audio(url=row[4]))
                    
                #widget spectrogram
                y, sr = librosa.load(path+wordInfo.motVedette+'_us.mp3')
                D = librosa.stft(y)  # STFT of y
                S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
                    
                        
                outSpec = widgets.Output()
                with outSpec:
                    fig, ax = plt.subplots()
                    img = librosa.display.specshow(S_db, x_axis='time', y_axis='linear', ax=ax)
                    ax.set(title='US spectrogram')
                    fig.colorbar(img, ax=ax, format="%+2.f dB")
                    plt.show(fig)
                    
                display(widgets.VBox([widgets.Label("US"), outAudio, outSpec])) 
                



button.on_click(on_button_clicked)

In [7]:
# définition du widget button permettant de lancer l'extraction
button = widgets.Button(description='Démarrer')

# définition du widget permettant l'affichage d'une sortie
out1 = widgets.Output()
out2 = widgets.Output()
button.on_click(on_button_clicked)

In [8]:
# les widgets de filtre input
vbFiltres = widgets.HBox([textTarget, ipaTarget, wid_nbsyll, wid_nbResult, button])
# les widgets d'output
vbOuts = widgets.VBox([cambridge, longman, linguee, tableau])

hbAllWidgets = widgets.VBox([vbFiltres, vbOuts])
hbAllWidgets

D:\Anaconda\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
D:\Anaconda\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
